# Overture Buildings


Welcome! This Notebook designed to perform a spatial query on the Overture Buildings dataset. 

The original dataset is several GB in size. The Fused User Defined Function (UDF) below fetches only the buildings that fall within the area of interest. This means you can use this code to simplify your workflows by loading only the fraction of data you care about.


<img src="https://fused-magic.s3.us-west-2.amazonaws.com/docs_assets/overture_spatial_query.png" alt="overture" width="600"/></a>


The Notebook shows how to:

1. Define an area of interest by drawing a polygon on a Leaflet map
2. Perform a spatial query over the dataset with a UDF
3. View the output on a map

It shows examples for using both `ipyleaflet` and `geemap`.

Let's begin!

In [ ]:
# Uncomment this line to install packages if needed
!pip3 install fused ipyleaflet geopandas

# 1. Load Fused UDF

This User Defined Function (UDF) queries a geo partitioned version of the Overture Buildings Dataset (hosted on an S3 bucket) that corresponds to the area of an input GeoDataFrame. It returns a GeoDataFrame containing the subsampled data, which gets cached to the local environment for added speed.

In [ ]:
import geopandas as gpd
from shapely.geometry import shape

import fused

# Create a map centered at the given location
MAP_LOCATION = (37.7749, -122.4194)

In [ ]:
udf = fused.load(
    "https://github.com/fusedio/udfs/tree/0905bef/public/Overture_Maps_Example"
)

# 2. Draw polygon to query buildings

## ipyleaflet

Draw a polygon around the AOI and on the map and wait for building polygons to load.

Note: the workflow is designed to work with a single polygon.

In [ ]:
from ipyleaflet import DrawControl, GeoData, Map, basemap_to_tiles, basemaps

m = Map(
    layers=(basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),),
    center=MAP_LOCATION,
    zoom=13,
)


def on_draw(self, action, geo_json):
    print("Running...")
    gdf = gpd.GeoDataFrame(
        index=[0], crs="epsg:4326", geometry=[shape(geo_json.get("geometry"))]
    )
    gdf_buildings = udf(bbox=gdf).run_local()
    m.add(GeoData(geo_dataframe=gdf_buildings))
    print("Done!")


draw_control = DrawControl()
draw_control.on_draw(on_draw)
draw_control.polygon = {
    "shapeOptions": {"color": "red", "fillOpacity": 0.0},
    "drawError": {"color": "#dd253b", "message": "Oups!"},
    "allowIntersection": False,
}

draw_control.rectangle = {"shapeOptions": {"color": "red", "fillOpacity": 0.0}}

m.add(draw_control)